<a href="https://colab.research.google.com/github/VisionLogic-AI/Brain_Projects/blob/master/CycleGAN_for_Cross_Modality_Image_Synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CycleGAN
What is CycleGAN?
CycleGAN is an unsupervised image to image architecture known for it's capacity for learning appings between classes of images without requiring paired data, making it something of a "universal image translator".

In [1]:
!wget -nc https://raw.githubusercontent.com/brainhack101/IntroDL/master/notebooks/2019/Eklund/requirements-gpu.txt

--2020-05-20 18:10:25--  https://raw.githubusercontent.com/brainhack101/IntroDL/master/notebooks/2019/Eklund/requirements-gpu.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90 [text/plain]
Saving to: ‘requirements-gpu.txt’

requirements-gpu.tx 100%[===================>]      90  --.-KB/s    in 0s      

2020-05-20 18:10:25 (5.53 MB/s) - ‘requirements-gpu.txt’ saved [90/90]



In [2]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [3]:
!pip install -r requirements-gpu.txt

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-r9svdly8
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-r9svdly8
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=9fb04536ef1c98b48964ea8223f74014eef633d608c6277177ac82a7d06e4690
  Stored in directory: /tmp/pip-ephem-wheel-cache-c9ds07k_/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [4]:
from keras.layers import Layer, Input, Dropout, Conv2D, Activation, add, UpSampling2D
from keras.layers import Conv2DTranspose, Flatten
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization, InputSpec
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Network

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from progress.bar import Bar
import datetime
import time
import json
import csv
import sys
import os

import keras.backend as K

Using TensorFlow backend.


In [5]:
!wget https://raw.githubusercontent.com/brainhack101/IntroDl/master/notebooks/2019/Eklund/helper_funcs.py

--2020-05-20 13:57:03--  https://raw.githubusercontent.com/brainhack101/IntroDl/master/notebooks/2019/Eklund/helper_funcs.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13105 (13K) [text/plain]
Saving to: ‘helper_funcs.py.2’

helper_funcs.py.2   100%[===================>]  12.80K  --.-KB/s    in 0s      

2020-05-20 13:57:03 (73.3 MB/s) - ‘helper_funcs.py.2’ saved [13105/13105]



In [0]:
from helper_funcs import *

If you have multiple GPU's, you can select a songle one of them by setting te visible CUDA device 0,1

In [0]:
import os
os.environ['CUDA_DEVICE_ORDER']= 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']= '0'

#Load Data

In [8]:
!wget -nc https://raw.githubusercontent.com/brainhack101/IntroDL/master/notebooks/2019/Eklund/data.zip
!unzip -uo data.zip

--2020-05-20 14:19:23--  https://raw.githubusercontent.com/brainhack101/IntroDL/master/notebooks/2019/Eklund/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 531636 (519K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 519.18K  --.-KB/s    in 0.07s   

2020-05-20 14:19:24 (7.56 MB/s) - ‘data.zip’ saved [531636/531636]

Archive:  data.zip
   creating: data/
   creating: data/malefemale/
   creating: data/malefemale/testA/
  inflating: data/malefemale/testA/im163.png  
  inflating: data/malefemale/testA/im188.png  
  inflating: data/malefemale/testA/im189.png  
  inflating: data/malefemale/testA/im175.png  
  inflating: data/malefemale/testA/im171.png  
  inflating: data/malefemale/testA/im170.png  
  inflating: data

In [0]:
image_folder= 'malefemale'
data= load_data(subfolder= image_folder)

#Model Parameters
This CycleGAN implementation allows a alot of freedom on both the training parameters and the network and network architecture.

In [0]:
opt= {}

#Data
opt['channels']= data['nr_of_channels']
opt['img_shape']= data['images_size'] + (opt['channels'], )
print('Image shape:', opt['img_shape'])

opt['A_Train']= data['trainA_images']
opt['B_Train']= data['trainB_images']
opt['A_test']= data['testA_images']
opt['B_test']= data['testB_images']
opt['testA_images_names']= data['testA_image_names']
opt['testB_images_names']= data['testB_image_names']

CycleGAN can be used on both paired and unpaired data.

In [0]:
opt['paired_data']= False

Training Parameters
 

In [0]:
#Training Parameters
opt['lambda_ABA']= 10.0   #cyclic loss weight A_2_B
opt['lambda_BAB']= 10.0  #cyclic loss weight B_2_A
opt['lmbda_adversarial']= 10.0    #weight for los from discriminator guess on synthetic images
opt['learning_rate_D']= 2e-4
opt['learning_rate_G']= 2e-4
opt['generator_iterations']= 3   #number of generator training iterations in each trainining loop
opt['discriminator_iterations']= 1   #number of discriminator traijning iterations in each training loop
opt['synthetic_pool_size']= 50   #size of image pools used for training the discriminators
opt['beta_1']= 0.5  #adam parameter
opt['beta_2']= 0.999  #adam parameter
opt['batch_size']= 10    #number of images per batch
opt['epochs']= 200    #choose multiples of 20 since the models are saved each 20th epoch

In [0]:
#Output parameters
opt['save_models']= True   #save or not the generator and discriminator models
opt['save-training_img']= True   #save or not example training results or only tmp.png
opt['save_training_img_interval']= 1   #number of epoch between saves of intermediate training results
opt['self.tmp_img_update_frequency']= 3   #number of batches between updates of tmp.pn

Architecture Parameters 

In [0]:
#Architecture parameters
opt['use_instance_normalization']= True    #use instance normalization or batch normalization
opt['use_dropout']= False  #dropout in residual blocks
opt['use_bias']= True   #use bias
opt['use_linear_decay']= True   #linear decay of learning rate, for both discriminators and generators
opt['decay_epoch']= 101   #the epoch where the linear decay of the learning rates start
opt['use_patchgun']= True   #PatchGAN - if false the discriminators learning rate should be decreased
opt['use_resize_convolution']= False    #resize convolution - instead of transpose convolution in deconvolution layers
opt['discriminator_sigmoid']= True    #add a final sigmoid activation to the disctiminator

#tweaks
opt['REAL_LABEL']= 1.0  #use e.g 0.9 to avoid training the discriminators to zero loss

#Model Architecture
Layer Blocks
These are the individual layer blocks that are used to build the generators and discriminator. 

In [0]:
#Discriminator Layers
def ck(model, opt, x, k, use_normalization, use_bias):
  x= Conv2D(filters= k, kernel_size=4, strides=2, padding= 'same', use_bias= use_bias)(x)
  if use_normalization:
    x= model['normalization'](axis= 3, center= True, epsilon= 1e-5)(x, training= True)
    x= LeakyReLU(alpha= 0.2)(x)
    return x

#first generator layer
def c7AK(model, opt, x, k):
  x= Conv2D(filters= k, kernel_size= 7, strides= 1, padding= 'valid', use_bias= opt['use_bias'])(x)
  x= model['normalization'](axis= 3, center= True, epsilon= 1e-5)(x, training= True)
  x= Activation('relu')(x)
  return x

#Downsampling
def dk(model, opt, x, k): 
  x= Conv2D(filters= k, kernel_size=3, strides=2, padding='same', use_bias= opt['use_bias'])(x)
  x= model['noralization'](axis= 3, centers = True, epsilon= 1e-5)(x, training= True)
  x= Activation('relu')(x)
  return x

#Residual block
def Rk(model, opt, x0):
  k= int(x0.shape[-1])

  #first layer
  x= ReflectionPadding2D((1,1))(x0)
  x= Conv2D(filters= k, kernel_size 3, strides=1, padding= 'valid', use_bias= opt['use_bias'])(x)
  x= model['noramlization'](axis=3, center= True, epsilon= 1e-5)(x, training= True)
  x= Activation('relu')(x)
  if opt['use_dropout']:
    x= Dropout(0.5)(x)

  #second layer
  x= ReflectionPadding2D((1,1))(x)
  x= Conv2D(filters= k, kernel_size= 3, strides= 1, padding= 'valid', use_bias= opt['use_bias'])(x)
  x = model['normalization'](axis=3, center= True, epsilon= 1e-5)(x, training= True)
  #Merge
  x= add([x, x0])

  return x

#UpSampling
def uk(model, opt, x, k):
  # up sampling followed by 1x1 convolution <=> fractional - strided 1/2
  if opt['use_resize_convolution']:
    x= UpSampling2D(size= (2,2))(x)    #nearest neighbor upsampling
    x= ReflectionPadding2D((1,1))(x)
    x= Conv2D(filters=k, kernel_size= 3, strides=1, padding= 'valid', use_bias= opt['use_bias'])(x)
  else:
    x= Conv2DTranspose(filters=k, kernel_size=3, strides=2, padding= 'same', use_bias= opt['use_bias'])(x)
  x= model['normalization'](axis= 3, center= True, epsilon= 1e-5)(x, training= True)
  x= Activation('relu')(x)
  return x

Architecture Functions

In [0]:
def build_discriminator(model, opt, name= None):
  #Input
  input_img= Input(shape= opt,['img_shape'])

  #Layers 1-4
  x= ck(model, opt, input_img, 64, False, True)    #instance normalization is not used for this layer
  x= ck(model, opt, x, 128, True, opt['use_bias'])
  x= ck(model, opt, x, 256, True, opt['use_bias'])
  x= ck(model, opt, x, 512, True, opt['use_bias'])

  #Layer 5: Output
  if opt['use_patchgan']:
    x= Conv2D(filters= 1, kernel_size= 4, strides=1, padding= 'same', use_bias= True)(x)
  else:
    x= Flatten()(x)
    x= Dense(1)(x)

  if opt['discriminator_sigmoid']:
    x= Activation('sigmoid')(x)

  return Model(inputs= input_img, outputs= x, name=name)

def build_generator(model, opt, name= None):
  #layer 1: Input
  input_img= Input(shape= opt['img_shape'])
  x= ReflectionPadding2D((3,3))(input_img)
  x= c7AK(model, opt, x, 32)

  #Layer 2-3: Downsampling
  x= dk(model, opt, x, 64)
  x= df(model, opt, x, 128)

  #Layers 4-12: Residual blocks
  for _ in range(4, 13):
    x= Rk(model, opt, x)

  #layer 13:14: UpSampling
  x= uk(model, opt, x, 64)
  x= uk(model, opt, x, 32)

  #layer 15: Output
  x= ReflectionPadding2D((3,3))(x)
  x= Conv2D(opt['channels'], kernel_size= 7, strides= 1, padding= 'valid', use_bias= True)(x)
  x= Activation('tanh')(x)

  return Model(inputs= input_img, outputs= x, name=name)

Loss Functions
The discriminators use MSE loss. The generators use MSE for the adversarial losses and MAE for the cycle consistency losses.

In [0]:
#Mean squared error
def mse(y_true, y_pred):
  loss= tf.reduce_mean(tf.squared_difference(y_pred, y_true))
  return loss

#mean absolute error
def mae(y_true, y_pred):
  loss= tf.reduce_mean(tf.abs(y_pred - y_true))
  return loss

Build CycleGAN Model

In [0]:
model= {}

#Normalization
model['normalization']= InstanceNormalization

#optimizers
model['opt_D']= Adam(opt['learning_rate_D'], opt['beta_1'], opt['beta_2'])
model['opt_G']= Adam(opt['learning_rate_G'], opt['beta_1'], opt['beta_2'])

#Build discrimiators
D_A= build_discriminator(model, opt, name= 'D_A')
D_B= build_discriminator(model, opt, name= 'D_B')

#define discrimiantor models
image_A= Input(shape= opt['img_shape'])
image_B= Input(shape= opt['img_shape'])
guess_A= D_A(image_A)
guess_B= D_B(image_B)
model['D_A']= Model(inputs= image_A, outputs= guess_A, name= 'D_A_model')
model['D_B']= Model(inputs= image_B, outputs= guess_B, name= 'D_B_model')

#Compile discriminator models
loss_weight_D= [0.5]    #0.5 since we train on real and synthetic images
model['D_A'].compile(optimizer= model['opt_D'],
                     loss= mse,
                     loss_weights= loss_weights_D)
model['D_B'].compile(optimizer= model['opt_D'],
                     loss= mse,
                     loss_weights= loss_weights_D)

#Use containers to make a static copy of discriminators, used when traiing the generators
model['D_A_static']= Network(inputs= image_A, outputs= guess_A, name= 'D_A_static_model')
model['D_B_static']= Network(inputs= image_B, outputs= guess_B, name= 'D_B_static_model')

#do not update the discriminator weights during the generator training
model['D_A_static'].trainable= False
model['D_B_static'].trainable= False

#Build generators
model['G_A2B']= build_generator(model, opt, name= 'G_A2B_model')
model['G_B2A']= build_generator(model, opt, name= 'G_B2A_model')

#Define full CycleGAN model, used for training the generators
real_A= Input(shape=opt['img_shape'], name= 'real_A')
real_B= Input(shape= opt['img_shape'], name= 'real_B')
synthetic_B= model['G_A2B'](real_A)
synthetic_A= model['G_B2A'](real_B)
dB_guess_synthetic= model['D_B_static'](synthetic_B)
dA_guess_synthetic= model['D_A_static'](synthetic_A)
reconstructed_A= model['G_B2A'](synthetic_B)
reconstructed_B= model['G_A2B'](synthetic_A)

#compile full cycleGAN model
model_outputs= [reconstructed_A, reconstructed_B,
                dB_guess_synthetic, dA_guess_synthetic]
compile_losses= [mae, mae, 
                 mse, mse]
compile_weights= [opt['lambda_ABA'], opt['lambda_BAB'],
                  opt['lambda_adversarial'], opt['lambda_adversarial']]
model['G_model']= Model(inputs= [real_A, real_B],
                        outputs= model_outputs,
                        name= 'G_model')
model['G_model'].compile(optimizer= model['opt_G'],
                         loss= compile_losses,
                         loss_weights= compile_weights)

Folders and configuration

In [0]:
opt['date_time']= time.strrftime('%Y%m%d-%H%M%S',time.localtime()) + '-' + image_folder

#Output folder for run data and images
opt['out_dir']= os.path.join('image', opt['date_time'])
if not os.path.exists(opt['out_dir']):
  os.makedirs(opt['out_dir'])

#Output folder for saved models
if opt['save_models']:
  opt['model_out_dir']= os.path.join('saved_models', opt['date_time'])
  if not os.path.exists(opt['model_out_dir']):
    os.makedirs(opt['model_out_dir'])

write_metadata_to_json(model, opt)

#Do not pre-allocate GPU memory, allocate as-needed
config= tf.ConfigProto()
config.gpu_options.allow_growth= True
K.tensorflow_backend.set_session(tf.Session(config= config))

#Train Function

In [0]:
def train(model, opt):
  
  def run_training_batch():

    #===================Discriminator Training=====================
    #Generate batch of synthetic images
    synthetic_images_B= model['G_A2B'].predict(real_images_A)
    synthetic_images_A= model['G_B2A'].predict(real_images_B)
    synthetic_images_B= synthetic_pool_B.query(synthetic_images_B)
    synthetic_images_A= synthetic_pool_A.query(synthetic_images_A)

    #Train discriminators on batch
    D_loss= []
    for _ in range(opt['discriminator_iterations']):
      D_A_loss_real= model['D_A'].train_on_batch(x= real_images_A, y= ones)
      D_B_loss_real= model['D_B'].train_on_batch(x= real_images_B, y= ones)
      D_A_loss_synthetic= model['D_A'].train_on_batch(x= synthetic_images_A, y= zeros)
      D_B_loss_synthetic= model['D_B'].train_on_batch(x= synthetic_images_B, y= zeros)
      D_A_loss= D_A_loss_real + D_A_loss_synthetic
      D_B_loss= D_B_loss_real + D_B_loss_synthetic
      D_loss.append(D_A_loss + D_B_loss)

    # ====== Generator Training ======
    target_data= [real_images_A, real_images_B, ones, ones]

    #train generators on batch
    G_Loss= []
    for _ in range(opt['generator_iterations']):
      G_loss.append(model['G_model'].train_on_batch(
          x=[real_images_A, real_images_B], y= target_data))
      
    #=====================================
    #update Learning rates
    if opt['use_linear_decay'] and epoch >= opt['decay_epoch']:
      update_lr(model['D_A'], decay_D)
      update_lr(model['D_B'], decay_D)
      update_lr(model['G_model'], decay_G)

    #Store training losses
    D_A_losses.append(D_A_loss)
    D_B_losses.append(D_B_loss)
    D_losses.append(D_loss[-1])

    ABA_reconstruction_loss= G_loss[-1][1]
    BAB_reconstruction_loss= G_loss[-1][2]
    reconstruction_loss= ABA_reconstruction_loss + BAB_reconstruction_loss
    G_AB_adversarial_loss= G_loss[-1][3]
    G_BA_adversarial_loss= G_loss[-1][4]

    ABA_reconstruction_losses.append(ABA_reconstruction_loss)
    BAB_reconstruction_losses.append(BAB_reconstruction_loss)
    reconstruction_losses.append(reconstruction_loss)
    G_AB_adversarial_losses.append(G_AB_adversarial_loss)
    G_BA_adversarial_losses.append(G_BA_adversarial_loss)
    G_losses.append9(G_loss[-1][0])

    #Print training status
    print('\n')
    print('Epoch ----------------------', epoch, '/', opt['epochs'])
    print('Loop index -----------------', loop_index + 1, '/', nr_im_per_epoch)
    if opt['discriminator_iterations']:
      print('Discriminator losses:')
      for i in range(opt['discriminator_iterations']):
        print('D_loss,' D_loss[i])
    if opt['generator_iterations'] > 1:
      print('Generator losses:')
      for i in range(opt['generator_iterarions']):
        print('G_loss', G_loss[i])
    print(' Summary:')
    print('D_lr', K.get_value(model['D_A'].optimizer.lr))
    print('G_lr', K.get_value(model['G_model'].optimizer.lr))
    print('D_loss:', D_loss[-1])
    print('G_loss:', G_loss[-1][0])
    print('reconstruction_loss:', reconstruction_loss)
    print_ETA(opt, start_time, epoch, nr_im_per_epoch, loop_index)
    sys.stdout.flush()

    if loop_index % 3*opt['batch_size']== 0:
      #save temporary images continously
      save_tmp_images(model, opt, real_images_A[0], real_images_B[0],
                      synthetic_images_A[0], synthetic_images_B[0])
      
    # ==================================================================
    #Begin Training
    # ==================================================================
    if opt['save_training_img'] and not os.path.exists(os.path.join(opt['out_dir'], 'train_A')):
      os.makedirs(os.path.join(opt['out_dir'], 'train_A'))
      os.makedirs(os.path.join(opt['out_dir'], 'train_B'))
      os.makedirs(os.path.join(opt['out_dir'], 'test_A'))
      os.makedirs(os.path.join(opt['out_dir'], 'test_B'))
    
    D_A_losses= []
    D_B_losses= []
    D_losses= []

    ABA_reconstruction_losses= []
    BAB_reconstruction_losses= []
    reconstruction_losses= []
    G_AB_adversarial_losses= []
    G_BA_adversarial_losses= []
    G_losses= []


    #Image pools used to update the discriminators
    synthetic_pool_A= ImagePool(opt['synthetic_pool_size'])
    synthetic_pool_B= ImagePool(opt['synthetic_pool_size'])

    #Labels used for discriminator training
    label_shape= (opt['batch_size'],) + model['D_A'].output_shape[1:]
    ones= np.ones(shape= label_shape) * opt['REAL_LABEL']
    zeros= ones * 0

    #linear learning rate decay
    if opt['use_linear_decay']:
      decay_D, decay_G= get_lr_linear_rate(opt)

    nr_train_im_A= opt['A_train'].shape[0]
    nr_train_im_B= opt['B_train'].shape[0]
    nr_im_per_epoch= int(np.ceil(np.max((nr_train_im_A, nr_train_im_B)) / opt['batch_size']) * opt['batch_size'])

    #Start stopwatch for ETA's
    start_time= time.time()
    timer_started= False

    for epoch in range(1, opt['epochs'] + 1):
      #random_order_A= np.random.randint(nr_train_im_A, size= nr_im_per_epoch)
      #random_order_B= np.random.randint(nr_train_im_B, size= nr_im_per_epoch)

      random_order_A= np.concatenate((np.random.permutation(nr_train_im_A),
                                      np.random.randint(nr_train_im_A, size= nr_im_per_epoch - nr_train_im_A)))
      random_order_B= np.concatenate((np.random.permutation(nr_train_im_B),
                                      np.random.randint(nr_train_im_B, size= nr_im_per_epoch - nr_train_im_B)))
      
      #Train on image batch
      for loop_index in range(0, nr_im_per_epoch, opt['batch_size']):
        indices_A= random_order_A[loop_index: loop_index + opt['batch_size']]
        indices_B= random_order_B[loop_index: loop_index + opt['batch_size']]

        real_images_A= opt['A_train'][indices_A]
        real_images_B= opt['B_train'][indices_B]

        #Train on image batch
        run_training_batch()

        #start timer after first (slow) iteration has finished
        if not timer_started:
          start_time= time.time()
          timer_started= True

      #Save training image,s
      if opt['save_training_img'] and epoch % opt['save_training_img_interval']== 0:
        print(' \n', '\n', ' ---------------------------- Saving images for epoch', epoch, '---------------------------', '\n', '\n')
        save_epoch_images(model, opt, epoch)

      #Save model
      if opt['save_models'] and epoch % 20==0:
        save_model(opt, model['D_A'], epoch)
        save_model(opt, model['D_B'], epoch)
        save_model(opt, model[G_A2B], epoch)
        save_model(opt, model['G_B2A'], epoch)


      #Save training history
      training_history= {
          'DA_losses': D_A_losses,
          'DB_losses': D_B_losses,
          'G_AB_adversarial_losses': G_AB_adversarial_losses,
          'G_BA_adversarial_losses': G_Ba_adversarial_losses,
          'ABA_reconstruction_losses': ABA_reconstruction_losses,
          'BAB_reconstruction_losses': BAB_reconstruction_losses,
          'reconstruction_losses': reconstruction_losses,
          'D_losses': D_losses,
          'G_losses': G_losses}
          write_loss_data_to_file(opt, training_history)